In [10]:
import librosa
import soundfile as sf
import matplotlib.pyplot as plt
from IPython.display import Audio
import pandas as pd
import numpy as np 
import os
import soundfile as sf
import time
from joblib import Parallel, delayed
from functools import partial
from multiprocessing import Pool

In [2]:
#get all features out of giver file 
def get_features(file_path, file_name):
    y, sr = librosa.load(file_path)
    iter_ = int(y.shape[0]/sr)
    features = []
    for i in range(iter_):
        y_seg = y[i*sr:(sr*i+sr)]
        chroma_stft = np.mean(librosa.feature.chroma_stft(y=y_seg, sr=sr))
        rms = np.mean(librosa.feature.rms(y=y_seg))
        spec_cent = np.mean(librosa.feature.spectral_centroid(y=y_seg, sr=sr))
        spec_bw = np.mean(librosa.feature.spectral_bandwidth(y=y_seg, sr=sr))
        rolloff = np.mean(librosa.feature.spectral_rolloff(y=y_seg, sr=sr))
        zcr = np.mean(librosa.feature.zero_crossing_rate(y_seg))
        mfcc = np.mean(librosa.feature.mfcc(y=y_seg, sr=sr, n_mfcc=20),axis=1)
        features.append([chroma_stft,rms,spec_cent,spec_bw,rolloff,zcr,*mfcc]) # starred expressions for mfcc to get 20 columns instead of 20 row
        # b = np.concatenate((b,a),axis=1) # ajout filename apres?
    
    return features
    

In [3]:
audio_dir='LA/'
# label = 'REAL'
folders = ['ASVspoof2019_LA_dev','ASVspoof2019_LA_eval','ASVspoof2019_LA_train']
sound_files_path = []
for folder in folders:
    files = os.listdir(os.path.join(audio_dir,folder,'flac'))
    for file in files:
        path = os.path.join(os.path.join(audio_dir,folder,'flac',file))
        if file.endswith('.flac'):
            sound_files_path.append(path)
len(sound_files_path) 
sound_files_path[0][28:41]

'LA_D_8848915.'

In [25]:
sound_files_path[0][28:40]

'LA_D_8848915'

In [20]:
#make dataframe with all the features
def make_features_df(audio_dir):
    feature_list = []
    for audio in audio_dir:
        filename = audio[28:40]
        file_features=get_features(audio,filename)
        for segment_features in file_features:
                feature_list.append(segment_features + [filename])

    features_df = pd.DataFrame(feature_list,columns = data_set.columns)
    # print(f'{filename} done')
    return features_df


In [ ]:
n_proc = 8
data_set = pd.read_csv('Data/DATASET-balanced.csv')

start = time.time()
r = Parallel(n_jobs=n_proc)(delayed(make_features_df)([sound]) for sound in sound_files_path)
print('Time multiprocessing with joblib :', time.time() - start)

# asvs_features = make_features_df(sound_files_path)

/opt/conda/lib/python3.10/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/opt/conda/lib/python3.10/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/opt/conda/lib/python3.10/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/opt/conda/lib/python3.10/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/opt/conda/lib/python3.10/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/opt/conda/lib/python3.10/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/opt/conda/lib/python3.10/site-packages/librosa/core/pitch.py:101: Use

In [29]:
169/10000*len(sound_files_path)/60

34.44755166666666

In [30]:
first_part = r

In [34]:
fulldf= pd.DataFrame(first_part[0])
for r in first_part[1:]:
    partialdf = pd.DataFrame(r)
    fulldf = pd.concat([fulldf,partialdf])
    

fulldf
    


,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,LABEL
0,0.581642,0.003070,2199.394712,1951.288198,4489.428156,0.101729,-465.421295,125.339874,-69.007561,70.611092,...,-8.866219,7.612215,-12.527738,9.243306,-17.393263,6.024668,-14.883923,-1.570087,-11.686265,LA_D_8848915
1,0.325574,0.093229,2345.454133,1599.777611,3984.376665,0.152854,-236.496674,100.673485,-32.507786,49.088181,...,-10.290752,-2.104480,-12.366513,-0.074342,-20.750038,-0.219439,-13.774378,-12.227905,-15.365830,LA_D_8848915
2,0.315612,0.107650,1792.799111,1426.987016,3165.870250,0.124190,-233.423737,128.119858,-33.989712,45.122196,...,-4.690394,-8.381293,-15.633381,-2.263502,-20.402868,-5.021841,-14.528839,-10.487343,-13.769796,LA_D_8848915
0,0.539373,0.001211,1397.669671,1798.662060,3206.245006,0.058805,-601.982971,157.632462,-29.607780,64.629547,...,-18.353567,-2.086941,-21.357689,1.418289,-19.528841,2.797719,-17.868378,-0.484152,-12.906013,LA_D_4477471
1,0.309486,0.177945,1369.867761,1431.153215,2583.005593,0.078857,-275.831085,104.976555,-20.991016,46.207661,...,-12.673950,-6.094771,-14.902205,0.231349,-26.601341,-16.148132,-9.064847,-9.427711,-17.687447,LA_D_4477471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.279929,0.224462,1434.757393,1212.123877,2442.305686,0.097456,-223.824463,133.702682,-31.633156,12.523908,...,-12.415925,-8.417245,-15.321552,2.185878,-26.737574,-16.954967,-7.676735,-7.728964,-16.519455,LA_D_1035474
0,0.551183,0.001504,1383.603525,1822.387476,3194.744318,0.054976,-582.024536,154.066269,-23.864571,63.886951,...,-18.274767,-2.316636,-20.569038,0.554391,-18.750593,2.466581,-16.960526,-1.237893,-12.460297,LA_D_8648680
1,0.367849,0.148790,1941.550486,1593.535645,3373.616721,0.122781,-261.024811,98.563309,-17.863111,46.333282,...,-13.559737,-3.721426,-18.178530,2.230018,-23.447760,-11.332749,-12.443802,-7.943543,-15.225898,LA_D_8648680
2,0.408558,0.120144,2058.191606,1595.348323,3679.730779,0.134433,-274.397736,77.170967,-21.059069,55.792919,...,-12.576080,-4.159424,-17.387138,1.516601,-17.670601,-9.305642,-8.529463,-5.143360,-12.887780,LA_D_8648680


In [35]:
fulldf.to_csv('Data/asvc_features_partial.csv',index=False)